In [ ]:
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')                                                                                            

Mounted at /content/drive


In [ ]:
# 进入项目（注意必须是单引号），并切换进mmdetection-2.24.1目录，开始运行代码
import os
os.chdir('/content/drive/MyDrive/data/document_parses/pdf_json')



## Part1

In [ ]:
!nvidia-smi


Sun Jul 31 09:13:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import os, json

# this finds our json files
word=[]
path_to_json =r'/content/drive/MyDrive/data/document_parses/pdf_json'

json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json


file = open('test.txt', mode='w',encoding='utf-8')

for index, js in enumerate(json_files):

    with open(os.path.join(path_to_json, js)) as json_file:

        json_text = json.load(json_file)

# here you need to know the layout of your json and each json has to have

# the same structure (obviously not the structure I have here)
        a=len(json_text['body_text'][0])
        title = json_text['metadata']['title']
        file.write(title)
        i = 0
        '''while i<a:
            
            body_text = json_text['body_text'][i]["text"]
            
            file.write(body_text)
            i=i+1  '''
          


           

# here I push a list of data into a pandas DataFrame at row given by 'index'

       

# now that we have the pertinent json data in our DataFrame let's look at it

print(word)


### 1.1 split()

In [ ]:
f = open(r"new1.txt",encoding="utf-8")
a = f.read().split()
f.close()
a

### 1.2 NLTK

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize
f = open(r"new1.txt",encoding="utf-8")
b=[]
for text in f.readlines():    
    a=word_tokenize(text)
    b.append(a)
f.close()
b

### 1.3 BPE

In [ ]:
import re, collections

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens



In [ ]:
def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        # 看vocabulary里面的token频率，相当于上面的code中的tokens去除freq为0的
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        # vocab和其对应的tokens
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def measure_token_length(token):
    
    # 如果token最后四个元素是 < / w >
    if token[-4:] == '</w>':
        # 那就返回除了最后四个之外的长度再加上1(结尾)
        return len(token[:-4]) + 1
    else:
        # 如果这个token里面没有结尾就直接返回当前长度
        return len(token)
    
# 如果vocabulary里面找不到要拆分的词，就根据已经有的token现拆
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    # base case，没词进来了，那拆的结果就是空的
    if string == '':
        return []
    # 已有的sorted tokens没有了，那就真的没这个词了
    if sorted_tokens == []:
        return [unknown_token] * len(string)

    # 记录拆分结果
    string_tokens = []
    
    # iterate over all tokens to find match
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        
        # 自定义一个正则，然后要把token里面包含句号的变成[.]
        token_reg = re.escape(token.replace('.', '[.]'))
        
        # 在当前string里面遍历，找到每一个match token的开始和结束位置，比如string=good，然后token是o，输出[(2,2),(3,3)]?
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        # if no match found in the string, go to next token
        if len(matched_positions) == 0:
            continue
        # 因为要拆分这个词，匹配上的token把这个word拆开了，那就要拿到除了match部分之外的substring，所以这里要拿match的start
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]
        substring_start_position = 0
        
        
        # 如果有匹配成功的话，就会进入这个循环
        for substring_end_position in substring_end_positions:
            # slice for sub-word
            substring = string[substring_start_position:substring_end_position]
            # tokenize this sub-word with tokens remaining 接着用substring匹配剩余的sorted token，因为刚就匹配了一个
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            # 先把sorted token里面匹配上的记下来
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        # tokenize the remaining string 去除前头的substring，去除已经匹配上的，后面还剩下substring_start_pos到结束的一段substring没看
        remaining_substring = string[substring_start_position:]
        # 接着匹配
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    else:
        # return list of unknown token if no match is found for the string
        string_tokens = [unknown_token] * len(string)
        
    return string_tokens

"""
EXAMPLE:
    token frequency dictionary before sorting: {'natural': 3, 'language':2, 'processing': 4, 'lecture': 4}
    sorted tokens: ['processing', 'language', 'lecture', 'natural']
    
INPUT:
    token_frequencies: Dict[str, int] # Counter for token frequency
    
OUTPUT:
    sorted_token: List[str] # Tokens sorted by length and frequency

"""
def sort_tokens(tokens_frequencies):
    # 对 token_frequencies里面的东西，先进行长度排序，再进行频次，sorted是从低到高所以要reverse
    sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item:(measure_token_length(item[0]),item[1]), reverse=True)
    
    # 然后只要tokens不要频次
    sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

    return sorted_tokens


In [ ]:
num_merges = 10000
for i in range(num_merges):
    pairs = get_stats(bpe)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    bpe = merge_vocab(best, bpe)

    tokens = get_tokens(bpe)
    

BPE result

In [ ]:
bpe=get_vocab('/content/drive/MyDrive/data/test/new1.txt')


In [ ]:
print(list(bpe.items())[:5])

[('r e c o m b i n a t i o n </w>', 946), ('e v e r y </w>', 5269), ('d a y </w>', 7620), ('a b u n d a n t </w>', 822), ('i n </w>', 700514)]


In [ ]:
print(list(bpe.items())[:5])

[('re co m b in ation</w>', 946), ('e ver y</w>', 5269), ('d a y</w>', 7620), ('ab un d an t</w>', 822), ('in</w>', 700514)]


In [ ]:
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(bpe)

#sort tokens by length and frequency
sorted_tokens = sort_tokens(tokens_frequencies)


In [ ]:
print(len(sorted_tokens))

1127


### Save Text


In [ ]:
f=open("result.txt","w")
 
for line in sorted_tokens:
    f.write(line+'\n')
f.close()